This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [1]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [2]:
import numpy as np
import pandas as pd

from pathlib import Path
from flip import utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, fisher


flip_base = Path(__flip_dir_path__)
data_path = flip_base / 'data'

In [3]:
grid = pd.read_parquet(data_path / "data_density.parquet")
grid_window = pd.read_parquet(data_path / "data_window_density.parquet")

coordinates_density = np.array([grid["ra"], grid["dec"], grid["rcom_zobs"]])

DataDensity = data_vector.Dens(grid.to_dict(orient='list'))

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmag_fiducial = 6.0

power_spectrum_dict_bias = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2]]}
power_spectrum_dict = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2],
                              [kmt, pmt * np.array(grid_window["window_mt"])],
                              [ktt, ptt]]}


In [4]:
size_batch=500_000
number_worker=8
covariance_fisher = covariance.CovMatrix.init_from_flip(
    "ravouxcarreres",
    "density",
    power_spectrum_dict,
    coordinates_density=coordinates_density,
    size_batch=size_batch,
    number_worker=number_worker,
    additional_parameters_values=(sigmag_fiducial,),
    variant="nobeta",
)

print("Covariance free parameters: ", covariance_fisher.free_par)

/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000009.61 ]: 01-23 02:38  root            INFO     Covariance matrix generated from flip with ravouxcarreres model in 4.73e+00 seconds


Covariance free parameters:  ['fs8', 'bs8']


In [5]:
fisher_properties = {
    "inversion_method": "inverse",
}


In [6]:
parameter_dict = {
    "bs8": 0.8,
    "fs8": 0.4,
}

Fisher = fisher.FisherMatrix.init_from_covariance(
    covariance_fisher,
    DataDensity,
    parameter_dict,
    fisher_properties=fisher_properties,
)

parameter_name_list, fisher_matrix = Fisher.compute_fisher_matrix()

error = np.sqrt(np.diag(np.linalg.pinv(fisher_matrix)))

In [7]:
for p, e in zip(parameter_name_list, error):
    print(f"Fisher error on {p} = {e:.4f}")

Fisher error on fs8 = 0.1337
Fisher error on bs8 = 0.0804
